_DeepDiff 用法介绍_

DeepDiff 核心模块

- DeepDiff:该模块通过递归方式比较两个字典、可迭代对象、字符串和其他对象的深度差异
- DeepSearch:该模块支持在对象中搜索对象
- Extract:该模块可以根据值抽取其 Key 的路径,反过来根据 Key 路径提取其值


In [1]:
from deepdiff import DeepDiff

# DeepDiff


## 完全相同


In [2]:
t1 = {1: 1, 2: 2, 3: 3}
t2 = t1
# 同一对象返回{}
print(DeepDiff(t1, t2))

{}


## type 不同


In [3]:
t1 = {1: 1, 2: 2, 3: 3}
t2 = {1: 1, 2: "2", 3: 3}
# 类型改变时会校验并返回差异（一个是int，一个str）
print(DeepDiff(t1, t2))

{'type_changes': {'root[2]': {'old_type': <class 'int'>, 'new_type': <class 'str'>, 'old_value': 2, 'new_value': '2'}}}


## 值不同


In [7]:
t1 = {1: 1, 5: 2, 3: 3}
t2 = {1: 1, 5: 4, 3: 3}
print(DeepDiff(t1, t2))

{'values_changed': {'root[5]': {'new_value': 4, 'old_value': 2}}}


## dict 数量不同


In [8]:
t1 = {1: 1, 2: 2, 3: 3}
t2 = {1: 1, 3: 3, 5: 5, 6: 6}
print(DeepDiff(t1, t2))

{'dictionary_item_added': [root[5], root[6]], 'dictionary_item_removed': [root[2]]}


## 属性

### verbose_level-差异详细程度


In [ ]:
t1 = {1: 1, 2: 2, 3: 3}
t2 = {1: 1, 3: 3, 5: 5, 6: 6}
# verbose_level=2可疑把key和value一起返
print(DeepDiff(t1, t2, verbose_level=2))

### get_deep_distance-差异值的深度值

In [21]:
print(DeepDiff(10.0, 10.1, get_deep_distance=True))

{'values_changed': {'root': {'new_value': 10.1, 'old_value': 10.0}}, 'deep_distance': 0.0014925373134328302}


### exclude_paths-排除待比较的路径

In [22]:
t1 = { "ingredients": ["no meat", "no eggs"]}
t2 = {"ingredients": ["veggies", "tofu"]}
# 指定ingredients这个路径不对比差异
print(DeepDiff(t1, t2, exclude_paths="root['ingredients']"))

{}


### 忽略

* ignore_string_type_changes=True ： 忽略字符串类型的差异
* ignore_numeric_type_changes=True：忽略数字类型的差异
* ignore_string_case=True：忽略字母大小写的差异
* truncate_datetime='minute'：时间的比较截止到分钟，只比较年月日小时

### ignore_order-忽略对比的顺序 

In [26]:
t1 = [1, 2, 3]
t2 = [1, 3, 2]
print(DeepDiff(t1, t2))
print(DeepDiff(t1, t2, ignore_order=True))

{'values_changed': {'root[1]': {'new_value': 3, 'old_value': 2}, 'root[2]': {'new_value': 2, 'old_value': 3}}}
{}


### Dynamic Ignore Order- 忽略指定情况的顺序

In [32]:
t1_order = {'a': [1, 2], 'b': [3, 4]}
t2_order = {'a': [2, 1], 'b': [4, 3]}
print(DeepDiff(t1_order, t2_order, ignore_order=True))
{}
# 设置只忽略a路径的顺序
def ignore_order_func(level):
    return 'a' in level.path()

print(DeepDiff(t1_order, t2_order, ignore_order_func=ignore_order_func))

{}
{'values_changed': {"root['b'][0]": {'new_value': 4, 'old_value': 3}, "root['b'][1]": {'new_value': 3, 'old_value': 4}}}


## 定制比较规则


In [18]:
from deepdiff.operator import BaseOperator


class CustomClass:
    def __init__(self, d: dict, inn_list: list):
        self.dict = d
        self.dict["inn_list"] = inn_list


custom1 = CustomClass(d=dict(a=1, b=2), inn_list=[1, 2, 3])
custom2 = CustomClass(d=dict(c=3, d=4), inn_list=[1, 2, 3, 2])
custom3 = CustomClass(d=dict(a=1, b=2), inn_list=[1, 2, 3, 4])


class ListMatchOperator(BaseOperator):
    def give_up_diffing(self, level, diff_instance):
        # 把list转换为set进行比较
        if set(level.t1.dict["inn_list"]) == set(level.t2.dict["inn_list"]):
            return True

In [19]:
print(
    DeepDiff(
        custom1, custom2, custom_operators=[ListMatchOperator(types=[CustomClass])]
    )
)

{}


In [20]:
print(
    DeepDiff(
        custom2, custom3, custom_operators=[ListMatchOperator(types=[CustomClass])]
    )
)

{'dictionary_item_added': [root.dict['a'], root.dict['b']], 'dictionary_item_removed': [root.dict['c'], root.dict['d']], 'values_changed': {"root.dict['inn_list'][3]": {'new_value': 4, 'old_value': 2}}}
